# <font color='blue'>Business Analytics</font>

# <font color='blue'>Capítulo 12 - Text Analytics</font>
## <font color='blue'>Mini-Projeto 6</font>
### <font color='blue'>Análise de Preço e Análise Textual do Noticiário Econômico Para Previsão de Ativos Financeiros</font>

![title](imagens/mini-projeto6.png)

## Definição do Problema

Leia o manual em pdf no Capítulo 12 do curso.

## Fonte de Dados

Leia o manual em pdf no Capítulo 12 do curso.

## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
#!pip install -q textblob

In [4]:
#!pip install -q xgboost

In [5]:
#!pip install -q lightgbm

In [6]:
#!pip install -q pandas_datareader

In [7]:
#!pip install -q pmdarima

In [8]:
#!pip install -q -U statsmodels

In [9]:
# Imports
import warnings
warnings.filterwarnings('ignore')
import os
import re
import nltk
import xgboost
import lightgbm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller, acf, pacf
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from pmdarima import auto_arima
from math import sqrt
from pandas_datareader.data import DataReader
from datetime import datetime

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Versions" --iversions

## Carregando os Dados

In [ ]:
# Carregando o dataset de texto de notícias
columns = ['Date', 'Category', 'News']
df_texto = pd.read_csv("dados/india-news-headlines.csv", names = columns)

In [ ]:
# Shape do dataframe
df_texto.shape

In [ ]:
# Visualizando amostra do dataset
df_texto.head()

In [ ]:
# Vamos remover a primeira linha pois é o cabeçalho do arquivo
df_texto.drop(0, inplace = True)

In [ ]:
# Vamos remover a coluna categoria pois não precisamos dela para nossa análise
df_texto.drop('Category', axis = 1, inplace = True)

In [ ]:
# Visualizando amostra do dataset
df_texto.head()

In [ ]:
# Carregamos agora o dataset de cotação de ações
df_numerico = pd.read_csv("dados/BSESN.csv")

In [ ]:
# Shape do dataframe
df_numerico.shape

In [ ]:
# Visualizando amostra do dataset
df_numerico.head()

In [ ]:
# Visualizando amostra do dataset
df_numerico.tail()

## Análise Exploratória e Limpeza dos Dados

> Vamos explorar primeiro o dataset de texto

In [ ]:
# Ajusta a coluna de data e visualiza info do dataset de texto
df_texto["Date"] = pd.to_datetime(df_texto["Date"], format = '%Y%m%d')
df_texto.info()

Vamos remover dos textos caracteres que não sejam letras.

In [ ]:
# Removendo caracteres
df_texto.replace("[^a-zA-Z']", " ", regex = True, inplace = True)
df_texto["News"].head()

Vamos agrupar o total de manchetes de notícias para cada dia, remover duplicatas e fazer o reset do índice.

In [ ]:
%%time
df_texto['News'] = df_texto.groupby(['Date']).transform(lambda x : ' '.join(x)) 
df_texto = df_texto.drop_duplicates() 
df_texto.reset_index(inplace = True, drop = True)

In [ ]:
# Visualiza uma amostra dos dados
df_texto.head()

In [ ]:
# Vamos checar se temos valores ausentes
df_texto.isnull().sum()

In [ ]:
# Shape do dataframe
df_texto.shape

> Exploramos agora o dataset numérico

In [ ]:
# Ajustamos a coluna de data
df_numerico["Date"] = pd.to_datetime(df_numerico["Date"])

In [ ]:
# Visualiza uma amostra dos dados
df_numerico.head()

In [ ]:
# Info do dataset
df_numerico.info()

In [ ]:
# Resumo estatístico
df_numerico.describe()

In [ ]:
# Checando por valores ausentes
df_numerico.isnull().sum()

In [ ]:
# Drop dos valores NA
df_numerico.dropna(inplace = True)

In [ ]:
# Checando por valores ausentes
df_numerico.isnull().sum()

In [ ]:
# Shape
df_numerico.shape

In [ ]:
# Plot
plt.figure(figsize = (20,10))
df_numerico['Close'].plot()
plt.ylabel('BSESN')

## Análise Numérica

## Plot da Média Móvel

In [ ]:
# Variável que usaremos no gráfico
close = df_numerico['Close']

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

In [ ]:
# Calcula as estatísticas móveis
ma = close.rolling(window = 50).mean()
std = close.rolling(window = 50).std()

In [ ]:
# Plot
plt.figure(figsize = (16,8))
df_numerico['Close'].plot(color = 'g', label = 'Close')
ma.plot(color = 'r', label = 'Média Móvel')
std.plot(label = 'Desvio Padrão Móvel')
plt.legend()

## Plot dos Retornos

In [ ]:
# Calcula os retornos (ganhos)
retornos = close / close.shift(1) - 1

In [ ]:
# Plot
plt.figure(figsize = (18,10))
retornos.plot(label = 'Retorno', color = 'g')
plt.title("Retornos")

Vamos dividir os dados em treino e teste.

In [ ]:
# Split em dados de treino e teste
dados_treino = df_numerico[:3000]
dados_teste = df_numerico[3000:]

In [ ]:
dados_treino.shape

In [ ]:
dados_teste.shape

## Testando a Estacionaridade

Uma série estacionária é o que em matemática costuma se chamar série convergente, ou seja, aquela que flutua em torno de uma mesma média ao longo do tempo.

Para criar um modelo de previsão de série temporal, precisamos validar a suposição de que a série é estacionária. Vamos criar uma função e testar em nossos dados.

In [ ]:
# Função para estar a eatacionaridade
def testa_estacionaridade(timeseries):

    # Calcula as estatísticas móveis
    rolmean = timeseries.rolling(20).mean()
    rolstd = timeseries.rolling(20).std()

    # Plot das estatísticas móveis
    plt.figure(figsize = (18,9))
    plt.plot(timeseries, color = 'blue', label = 'Original')
    plt.plot(rolmean, color = 'r', label = 'Média Móvel')
    plt.plot(rolstd, color = 'black', label = 'Desvio Padrão Móvel')
    plt.xlabel('Data')
    plt.legend()
    plt.title('Estatísticas Móveis',  fontsize = 30)
    plt.show(block = False)
 
    print('Resultados do Teste Dickey Fuller:')
    result = adfuller(timeseries, autolag = 'AIC')
    labels = ['ADF Test Statistic', 'Valor-p', 'Número de Lags', 'Número de Observações']
    
    for value,label in zip(result, labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("Evidências fortes para rejeitar a hipótese nula (H0). Os dados são estacionários.")
    else:
        print("Evidência fraca contra hipótese nula. Série temporal não é estacionária.")

In [ ]:
# Aplica a função
testa_estacionaridade(dados_treino['Close'])

Série temporal não é estacionária. Vamos aplicar transformação de log aos dados.

In [ ]:
# Aplica transformação de log
dados_treino_log = np.log(dados_treino['Close']) 
dados_teste_log = np.log(dados_teste['Close'])

In [ ]:
# Média móvel
media_movel = dados_treino_log.rolling(24).mean() 

In [ ]:
# Plot
plt.figure(figsize = (20,10))
plt.plot(dados_treino_log) 
plt.plot(media_movel, color = 'red')

In [ ]:
# Drop de valores NA
dados_treino_log.dropna(inplace = True)
dados_teste_log.dropna(inplace = True)

In [ ]:
# Aplica a função
testa_estacionaridade(dados_treino_log)

Série ainda não estacionária. Vamos visualizar por outro ângulo.

In [ ]:
# Calcula a diferença entre log e media_movel
dados_treino_log_diff = dados_treino_log - media_movel
dados_treino_log_diff.dropna(inplace = True)

In [ ]:
# Aplcia a função
testa_estacionaridade(dados_treino_log_diff)

Sim, a série é estacionária. Vamos criar um modelo ARIMA.

In [ ]:
# Encontra o melhor modelo ARIMA, cria o modelo e faz as previsões
modelo = auto_arima(dados_treino_log, trace = True, error_action = 'ignore', suppress_warnings = True)
modelo.fit(dados_treino_log)
previsoes = modelo.predict(n_periods = len(dados_teste))
previsoes = pd.DataFrame(previsoes, index = dados_teste_log.index, columns = ['Prediction'])

In [ ]:
# Plot
plt.figure(figsize = (18,9))
plt.plot(dados_treino_log, label = 'Dados de Treino')
plt.plot(dados_teste_log, label = 'Dados de Teste')
plt.plot(previsoes, label = 'Previsões')
plt.title('Previsões de Ativos Financeiros')
plt.xlabel('Tempo')
plt.ylabel('Valor Real do Ativo Financeiro')

Calculamos o erro do modelo.

In [ ]:
# Calcula o erro
rms = np.sqrt(mean_squared_error(dados_teste_log, previsoes))
print("RMSE : ", rms)

## Análise Textual

Leia o manual em pdf no Capítulo 12.

In [ ]:
# Função para obter subjetividade
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [ ]:
# Função para obter polaridade
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
# Adicionando ao dataframe
df_texto['Subjectivity'] = df_texto['News'].apply(getSubjectivity)
df_texto['Polarity'] = df_texto['News'].apply(getPolarity)

In [ ]:
# Visualiza
df_texto.head()

In [ ]:
# Criando um analisador de sentimento
analisador = SentimentIntensityAnalyzer()

In [ ]:
%%time
df_texto['Compound'] = [analisador.polarity_scores(v)['compound'] for v in df_texto['News']]
df_texto['Negative'] = [analisador.polarity_scores(v)['neg'] for v in df_texto['News']]
df_texto['Neutral']  = [analisador.polarity_scores(v)['neu'] for v in df_texto['News']]
df_texto['Positive'] = [analisador.polarity_scores(v)['pos'] for v in df_texto['News']]

In [ ]:
# Visualiza
df_texto.head()

## Merge dos Dados Numéricos e de Texto

In [ ]:
# Visualiza
df_numerico.head()

In [ ]:
# Visualiza
df_texto.head()

In [ ]:
# Vamos concatenar os dataframe pela coluna de data
df_merge = pd.merge(df_numerico, df_texto, how = 'inner', on = 'Date')

In [ ]:
# Visualiza
df_merge.head()

## Criando os Datasets Para Treino dos Modelos

In [ ]:
# Dataframe com as variáveis que serão usadas para treinar os modelos
df_merge1 = df_merge[['Close', 'Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral', 'Positive']]

In [ ]:
# Visualiza
df_merge1.head()

> Os dataframes abaixo serão usados para o modelo de série temporal.

In [ ]:
# Colocamos a variável que é a série temporal em outro dataframe
df_time_series = df_merge1['Close']

In [ ]:
# Variáveis exógenas
exog_data = df_merge1[['Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral', 'Positive']]
exog_data

In [ ]:
# Variáveis de treino
final_train_data = df_time_series[:3000]
exog_train_data = exog_data[:3000]

In [ ]:
# Variáveis de teste
final_test_data = df_time_series[3000:]
exog_test_data = exog_data[3000:]

> Os dataframes abaixo serão usados para os modelos supervisionados.

In [ ]:
# Prepara X (variáveis de entrada)
X = df_merge1.drop('Close', axis = 1)
X

In [ ]:
# Prepara Y (variável de saída)
Y = df_merge1['Close']
Y

In [ ]:
# Divisão em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(X, Y, test_size = 0.2, random_state = 0)

## Modelo de Previsão de Série Temporal

In [ ]:
# Modelo de série temporal
modelo_v1 = SARIMAX(final_train_data, exog = exog_train_data, order = (1, 1, 1), seasonal_order = (0, 0, 0, 0))

In [ ]:
# Treinamento do modelo
resultado = modelo_v1.fit()

In [ ]:
resultado.summary()

In [ ]:
# Vamos fazer a previsão dos próximos 10 dias
forecast = resultado.forecast(10, exog = exog_test_data[:10])

In [ ]:
# Forecast
forecast

## Modelagem com Modelos Supervisionados

Vamos padronizar os dados e então treinar os modelos.

In [ ]:
x_treino.shape

In [ ]:
x_treino.head()

In [ ]:
# Cria o scaler para padronizar os dados
scaler = MinMaxScaler()

In [ ]:
# Treinamento do scaler
modelo_scaler = scaler.fit(x_treino)

In [ ]:
# Aplica em treino
x_treino = modelo_scaler.transform(x_treino)

In [ ]:
# Aplica em teste
x_teste = modelo_scaler.transform(x_teste)

In [ ]:
x_treino[10]

In [ ]:
x_teste[10]

In [ ]:
y_treino

In [ ]:
y_teste

## Modelo RandomForestRegressor

In [ ]:
# Cria o modelo 
modelo_v2 = RandomForestRegressor()

In [ ]:
# Treinamento
modelo_v2.fit(x_treino, y_treino)

In [ ]:
# Previsões
pred_v2 = modelo_v2.predict(x_teste)

In [ ]:
print('Root Mean Squared Error: ', sqrt(mean_squared_error(pred_v2, y_teste)))

## Modelo LGBMRegressor

In [ ]:
# Cria o modelo
modelo_v3 = lightgbm.LGBMRegressor()

In [ ]:
# Treinamento
modelo_v3.fit(x_treino, y_treino)

In [ ]:
# Previsões
pred_v3 = modelo_v3.predict(x_teste)

In [ ]:
print('Root Mean Squared Error: ', sqrt(mean_squared_error(pred_v3, y_teste)))

## Modelo XGBRegressor

In [ ]:
# Cria o modelo
modelo_v4 = xgboost.XGBRegressor()

In [ ]:
# Treinamento
modelo_v4.fit(x_treino, y_treino)

In [ ]:
# Previsões
pred_v4 = modelo_v4.predict(x_teste)

In [ ]:
print('Root Mean Squared Error: ', sqrt(mean_squared_error(pred_v4, y_teste)))

## Conclusão

Construímos 4 versões do modelo preditivo usando diferentes abordagens. Todos os modelos tem espaço para melhorias, seja no pré-processamento de dados ou na otimização de hiperparâmetros.

Um ponto de observação sobre este projeto é que para usá-lo em produção seria necessário um Engenheiro de Dados para construir todo o pipeline de dados até alimentar o modelo treinado.

Exemplo:

- Para o modelo SARIMAX imagine que tenhamos dados até 30 de Junho e queremos prever o valor do ativo financeiro em 10 de Julho. Treinamos o modelo com dados históricos e seria necessário ter o noticiário econômico de 1 a 9 de Julho e realizar a análise de sentimento, para então usar como variáveis exógenas e prever o próximo item da série.

- Para os modelos tradicionais podemos usar dados históricos de análise de sentimento e prever o valor do ativo financeiro independente da data.

Ambas as opções são quase previsão em tempo real e vão requerer um bom trabalho de Engenharia de Dados para construir o pipeline (sequência ou fluxo) dos dados. Nada impede que usemos dados históricos de análise de sentimento em ambos os casos.

# Fim